### Handling Imbalanced Dataset with Machine Learning

In [1]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings(action="ignore")

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report

from imblearn.under_sampling import NearMiss
from imblearn.over_sampling import RandomOverSampler
from imblearn.combine import SMOTEENN
from imblearn.ensemble import EasyEnsembleClassifier
from collections import Counter

In [2]:
df = pd.read_csv("creditcard.csv")

In [3]:
df.shape

(284807, 31)

In [4]:
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [5]:
df.isna().sum()

Time      0
V1        0
V2        0
V3        0
V4        0
V5        0
V6        0
V7        0
V8        0
V9        0
V10       0
V11       0
V12       0
V13       0
V14       0
V15       0
V16       0
V17       0
V18       0
V19       0
V20       0
V21       0
V22       0
V23       0
V24       0
V25       0
V26       0
V27       0
V28       0
Amount    0
Class     0
dtype: int64

In [6]:
df["Class"].value_counts()

0    284315
1       492
Name: Class, dtype: int64

In [7]:
# Independent and Dependent features
X = df.drop(columns="Class")
y = df["Class"]

### Train,Test and Split

In [8]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3)

### Using Logistic Regression with Hyperparameter tuning

In [9]:
log_reg = LogisticRegression()

In [10]:
grid = {"C":10.0**np.arange(0,3),"penalty":["l1","l2"]}
grid

{'C': array([  1.,  10., 100.]), 'penalty': ['l1', 'l2']}

In [11]:
cv = KFold(n_splits = 5,shuffle=False,random_state=None)

In [12]:
clf = GridSearchCV(log_reg,param_grid=grid,cv=cv,scoring="f1_macro")

In [13]:
clf.fit(X_train,y_train)

GridSearchCV(cv=KFold(n_splits=5, random_state=None, shuffle=False),
             estimator=LogisticRegression(),
             param_grid={'C': array([  1.,  10., 100.]),
                         'penalty': ['l1', 'l2']},
             scoring='f1_macro')

In [14]:
y_pred = clf.predict(X_test)

In [15]:
confusion_matrix(y_test,y_pred)

array([[85236,    49],
       [   49,   109]], dtype=int64)

In [16]:
accuracy_score(y_test,y_pred)

0.9988530365272755

In [17]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     85285
           1       0.69      0.69      0.69       158

    accuracy                           1.00     85443
   macro avg       0.84      0.84      0.84     85443
weighted avg       1.00      1.00      1.00     85443



### Using Random Forest Classifier

In [18]:
rf = RandomForestClassifier()
rf.fit(X_train,y_train)

RandomForestClassifier()

In [19]:
y_pred = rf.predict(X_test)

In [20]:
confusion_matrix(y_test,y_pred)

array([[85278,     7],
       [   31,   127]], dtype=int64)

In [21]:
accuracy_score(y_test,y_pred)

0.9995552590615966

In [22]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     85285
           1       0.95      0.80      0.87       158

    accuracy                           1.00     85443
   macro avg       0.97      0.90      0.93     85443
weighted avg       1.00      1.00      1.00     85443



In [23]:
X_train.shape

(199364, 30)

In [24]:
y_train.value_counts()

0    199030
1       334
Name: Class, dtype: int64

### Using class_weight parameter in Random Forest Classifier

In [25]:
class_weight = {0:1,1:100}

In [26]:
rf = RandomForestClassifier(class_weight=class_weight)
rf.fit(X_train,y_train)

RandomForestClassifier(class_weight={0: 1, 1: 100})

In [27]:
y_pred = rf.predict(X_test)

In [28]:
confusion_matrix(y_test,y_pred)

array([[85282,     3],
       [   37,   121]], dtype=int64)

In [29]:
accuracy_score(y_test,y_pred)

0.9995318516437859

In [30]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     85285
           1       0.98      0.77      0.86       158

    accuracy                           1.00     85443
   macro avg       0.99      0.88      0.93     85443
weighted avg       1.00      1.00      1.00     85443



# Under Sampling

### Near Miss

In [31]:
y_train.value_counts()

0    199030
1       334
Name: Class, dtype: int64

In [32]:
ns = NearMiss(0.8)
X_train_ns,y_train_ns = ns.fit_resample(X_train,y_train)

In [33]:
print("Number of classes before fit()",Counter(y_train))
print("Number of classes after fit()",Counter(y_train_ns))

Number of classes before fit() Counter({0: 199030, 1: 334})
Number of classes after fit() Counter({0: 417, 1: 334})


In [34]:
rf = RandomForestClassifier()
rf.fit(X_train_ns,y_train_ns)

RandomForestClassifier()

In [35]:
y_pred = rf.predict(X_test)

In [36]:
confusion_matrix(y_test,y_pred)

array([[57523, 27762],
       [   10,   148]], dtype=int64)

In [37]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       1.00      0.67      0.81     85285
           1       0.01      0.94      0.01       158

    accuracy                           0.67     85443
   macro avg       0.50      0.81      0.41     85443
weighted avg       1.00      0.67      0.80     85443



# Over Sampling

### Random Over Sampler

In [38]:
y_train.value_counts()

0    199030
1       334
Name: Class, dtype: int64

In [39]:
rs = RandomOverSampler(0.5)
X_train_rs,y_train_rs = rs.fit_resample(X_train,y_train)

In [40]:
print("Number of classes before fit()",Counter(y_train))
print("Number of classes after fit()",Counter(y_train_rs))

Number of classes before fit() Counter({0: 199030, 1: 334})
Number of classes after fit() Counter({0: 199030, 1: 99515})


In [41]:
rf = RandomForestClassifier()
rf.fit(X_train_rs,y_train_rs)

RandomForestClassifier()

In [42]:
y_pred = rf.predict(X_test)

In [43]:
confusion_matrix(y_test,y_pred)

array([[85282,     3],
       [   33,   125]], dtype=int64)

In [44]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     85285
           1       0.98      0.79      0.87       158

    accuracy                           1.00     85443
   macro avg       0.99      0.90      0.94     85443
weighted avg       1.00      1.00      1.00     85443



In [45]:
rs = RandomOverSampler(0.75)
X_train_rs,y_train_rs = rs.fit_resample(X_train,y_train)

In [46]:
print("Number of classes before fit()",Counter(y_train))
print("Number of classes after fit()",Counter(y_train_rs))

Number of classes before fit() Counter({0: 199030, 1: 334})
Number of classes after fit() Counter({0: 199030, 1: 149272})


In [47]:
rf = RandomForestClassifier()
rf.fit(X_train_rs,y_train_rs)

RandomForestClassifier()

In [48]:
y_pred = rf.predict(X_test)

In [49]:
confusion_matrix(y_test,y_pred)

array([[85280,     5],
       [   35,   123]], dtype=int64)

In [50]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     85285
           1       0.96      0.78      0.86       158

    accuracy                           1.00     85443
   macro avg       0.98      0.89      0.93     85443
weighted avg       1.00      1.00      1.00     85443



### SMOTEENN

In [51]:
sm = SMOTEENN(0.75)
X_train_sm,y_train_sm= sm.fit_resample(X_train,y_train)

In [52]:
print("Number of classes before fit()",Counter(y_train))
print("Number of classes after fit()",Counter(y_train_sm))

Number of classes before fit() Counter({0: 199030, 1: 334})
Number of classes after fit() Counter({0: 183073, 1: 140811})


In [53]:
rf = RandomForestClassifier()
rf.fit(X_train_sm,y_train_sm)

RandomForestClassifier()

In [54]:
y_pred = rf.predict(X_test)

In [55]:
confusion_matrix(y_test,y_pred)

array([[85266,    19],
       [   25,   133]], dtype=int64)

In [56]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     85285
           1       0.88      0.84      0.86       158

    accuracy                           1.00     85443
   macro avg       0.94      0.92      0.93     85443
weighted avg       1.00      1.00      1.00     85443



### Ensemble Techniques

In [57]:
easy = EasyEnsembleClassifier()
easy.fit(X_train,y_train)

EasyEnsembleClassifier()

In [58]:
y_pred = easy.predict(X_test)

In [59]:
confusion_matrix(y_test,y_pred)

array([[81898,  3387],
       [   14,   144]], dtype=int64)

In [60]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       1.00      0.96      0.98     85285
           1       0.04      0.91      0.08       158

    accuracy                           0.96     85443
   macro avg       0.52      0.94      0.53     85443
weighted avg       1.00      0.96      0.98     85443

